# Data Cleaning and Preparation

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [3]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

In [4]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [5]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
print(data)
print(cleaned)

     0    1    2
0  1.0  6.5  3.0
1  1.0  NaN  NaN
2  NaN  NaN  NaN
3  NaN  6.5  3.0
     0    1    2
0  1.0  6.5  3.0


In [7]:
#Remove a coluna que contém apenas valores NaN
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [8]:
data[2][0] = NA
data[2][3] = NA
data
data.dropna(axis=1, how='all')

,0,1
0,1.0,6.5
1,1.0,NaN
2,NaN,NaN
3,NaN,6.5


In [9]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [10]:
#Mantém linhas com pelo menos 2 NaN
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling In Missing Data

In [11]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [12]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [13]:
#O metodo fill sempre retorna um novo DataFrame, mas com o inplace=True ele altera também o original
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [16]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.332883,-2.359419,-0.199543
1,-1.541996,-0.970736,-1.307030
2,0.286350,NaN,-0.753887
3,0.331286,NaN,0.069877
4,0.246674,NaN,NaN
5,1.327195,NaN,NaN


In [18]:
#pad / ffill: propaga a última observação válida para a frente para o próximo 
#backfill / bfill válido: use a próxima observação válida para preencher a lacuna
df.fillna(method='ffill')

,0,1,2
0,0.332883,-2.359419,-0.199543
1,-1.541996,-0.970736,-1.307030
2,0.286350,-0.970736,-0.753887
3,0.331286,-0.970736,0.069877
4,0.246674,-0.970736,0.069877
5,1.327195,-0.970736,0.069877


In [19]:
#Número máximo de elementos seguidos que serão preenchidos.
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.332883,-2.359419,-0.199543
1,-1.541996,-0.970736,-1.307030
2,0.286350,-0.970736,-0.753887
3,0.331286,-0.970736,0.069877
4,0.246674,NaN,0.069877
5,1.327195,NaN,0.069877


In [20]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

### Removing Duplicates

In [21]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [22]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [23]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [24]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [25]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [26]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [29]:
data[data['v1'] > 3]
data.query('v1 > 3')
# Atribuição
#data.query('v1 > 3')[:] = 10

,k1,k2,v1
4,one,3,4
5,two,4,5
6,two,4,6


### Transforming Data Using a Function or Mapping

In [33]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [34]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}
meat_to_animal

{'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'}

In [35]:
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [36]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

In [37]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [38]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [39]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [40]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [41]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [43]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [44]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [45]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [46]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [47]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [48]:
# Com o inplace é alterado o DataFrame atual
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

In [49]:
#Dados contínuos com frequência são discretizados ou, 
#de modo alternativo, separados em “compartimentos” (bins) 
#para análise. Suponha que tenhamos dados sobre um grupo de 
#pessoas em um estudo e queremos agrupá-las em conjuntos de idades discretas:

ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [50]:
#O objeto devolvido pelo pandas é um objeto Categorical especial. 
#A saída que vemos descreve os compartimentos calculados pelo 
#pandas.cut. Podemos tratá-la como um array de strings informando 
#o nome do compartimento; internamente, ela contém um array categories 
#que especifica os nomes distintos das categorias, junto com rótulos 
#para os dados de ages no atributo codes:

bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [51]:
#Codes are an array of integers which are the positions of the actual values in the categories array.
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [52]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [53]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [54]:
#De forma consistente com a notação matemática para 
#intervalos, um parêntese significa que o lado está 
#aberto, enquanto o colchete indica que está fechado 
#(é inclusivo). Podemos alterar o lado que está fechado
# passando right=False:

pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [55]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [56]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.11, 0.32], (0.74, 0.95], (0.32, 0.53], (0.74, 0.95], (0.74, 0.95], ..., (0.74, 0.95], (0.74, 0.95], (0.74, 0.95], (0.11, 0.32], (0.74, 0.95]]
Length: 20
Categories (4, interval[float64]): [(0.11, 0.32] < (0.32, 0.53] < (0.53, 0.74] < (0.74, 0.95]]

In [57]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(-0.0309, 0.613], (0.613, 3.928], (-0.686, -0.0309], (-0.686, -0.0309], (-2.9499999999999997, -0.686], ..., (-2.9499999999999997, -0.686], (-2.9499999999999997, -0.686], (-0.686, -0.0309], (0.613, 3.928], (-2.9499999999999997, -0.686]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.686] < (-0.686, -0.0309] < (-0.0309, 0.613] < (0.613, 3.928]]

In [58]:
pd.value_counts(cats)

(0.613, 3.928]                   250
(-0.0309, 0.613]                 250
(-0.686, -0.0309]                250
(-2.9499999999999997, -0.686]    250
dtype: int64

In [59]:
cats2 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats2

[(-0.0309, 1.287], (-0.0309, 1.287], (-1.187, -0.0309], (-1.187, -0.0309], (-1.187, -0.0309], ..., (-1.187, -0.0309], (-1.187, -0.0309], (-1.187, -0.0309], (1.287, 3.928], (-2.9499999999999997, -1.187]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0309] < (-0.0309, 1.287] < (1.287, 3.928]]

In [60]:
pd.value_counts(cats2)

(-0.0309, 1.287]                 400
(-1.187, -0.0309]                400
(1.287, 3.928]                   100
(-2.9499999999999997, -1.187]    100
dtype: int64

### Detecting and Filtering Outliers
Detectando e filtrando valores discrepantes

In [61]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.046794,0.025429,-0.006855,-0.047439
std,0.997575,1.009162,0.993734,0.997187
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.599807,-0.612162,-0.697084,-0.743886
50%,0.043663,-0.011070,-0.029924,-0.086309
75%,0.746527,0.695298,0.694459,0.624413
max,2.653656,3.525865,2.735527,3.366626


In [62]:
#Suponha que quiséssemos encontrar os valores que excedessem 3 em valor absoluto em uma das colunas:
col = data[2]
col[np.abs(col) > 3]

37    -3.399312
132   -3.745356
Name: 2, dtype: float64

In [64]:
#Para selecionar todas as linhas que tenham um valor que exceda 3 ou –3, podemos utilizar o método any em um DataFrame booleano:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
37,0.457246,-0.025907,-3.399312,-0.974657
56,1.951312,3.260383,0.963301,1.201206
132,0.508391,-0.196713,-3.745356,-1.520113
231,-0.242459,-3.056990,1.918403,-0.578828
254,0.682841,0.326045,0.425384,-3.428254
318,1.179227,-3.184377,1.369891,-1.074833
540,-3.548824,1.553205,-2.186301,1.277104
631,-0.578093,0.193299,1.397822,3.366626
778,-0.207434,3.525865,0.283070,0.544635
799,-3.645860,0.255475,-0.549574,-1.907459


In [65]:
#Valores podem ser definidos com base nesses critérios. 
#Eis um código para eliminar os valores que estejam fora do intervalo de –3 a 3:

data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.047988,0.024884,-0.005711,-0.047378
std,0.993553,1.005924,0.989915,0.994634
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.599807,-0.612162,-0.697084,-0.743886
50%,0.043663,-0.011070,-0.029924,-0.086309
75%,0.746527,0.695298,0.694459,0.624413
max,2.653656,3.000000,2.735527,3.000000


In [66]:

np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,1.0,-1.0,1.0,-1.0
2,1.0,-1.0,1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0
4,1.0,1.0,-1.0,1.0


### Permutation and Random Sampling

In [68]:
#Permutar (reordenar aleatoriamente) uma Series ou as linhas 
#de um DataFrame é fácil utilizando a função numpy.random.permutation. 
#Chamar permutation com o tamanho do eixo que você quer permutar 
#gera um array de inteiros informando a nova ordem:

df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [69]:
sampler = np.random.permutation(5)
sampler

array([4, 1, 0, 3, 2])

In [71]:
df
df.take(sampler)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11


In [72]:
#Para selecionar um subconjunto aleatório sem substituição, o método sample pode ser usado em Series e em DataFrame:
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7


In [73]:
choices = pd.Series([5, 7, -1, 6, 4])
print(choices)
draws = choices.sample(n=10, replace=True)
print(draws)

0    5
1    7
2   -1
3    6
4    4
dtype: int64
3    6
0    5
1    7
0    5
4    4
3    6
1    7
1    7
0    5
1    7
dtype: int64


### Computing Indicator/Dummy Variables
Calculando variáveis indicadoras/dummy

Outro tipo de transformação para modelagem estatística ou aplicações de aprendizado de máquina (machine learning) consiste em converter uma variável de categorias em uma matriz “dummy” ou “indicadora”. Se uma coluna em um DataFrame tiver k valores distintos, poderíamos derivar uma matriz ou um DataFrame com k colunas contendo somente 1s e 0s. O pandas tem uma função get_dummies para isso, embora criar uma função por conta própria não seria difícil.

In [74]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [75]:
df['key']

0    b
1    b
2    a
3    c
4    a
5    b
Name: key, dtype: object

In [76]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [77]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [80]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [81]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [82]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [83]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

In [86]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]
movies_windic[movies_windic["Genre_Children's"] == 1]

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children's,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,13,Balto (1995),Animation|Children's,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,34,Babe (1995),Children's|Comedy|Drama,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,3775,Make Mine Music (1946),Animation|Children's|Musical,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3707,3776,Melody Time (1948),Animation|Children's|Musical,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3730,3799,Pok�mon the Movie 2000 (2000),Animation|Children's,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3750,3820,Thomas and the Magic Railroad (2000),Children's,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
#Criando uma matriz Dummies a partir da discretização de valores com o CUT
np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation

### String Object Methods

In [91]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [92]:
#TRIM
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [93]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [94]:
'::'.join(pieces)

'a::b::guido'

In [95]:
'guido' in val
val.index(',')
val.find(':')

-1

In [96]:
val.index(':')

ValueError: substring not found

In [97]:
val.count(',')

2

In [98]:
val.replace(',', '::')
val.replace(',', '')

'ab  guido'

| Argumento | | Descrição |
|-----------|-|-----------|
|count| | Devolve o número de ocorrências de uma substring na string, sem sobreposição.|
|endswith| |Devolve True se a string terminar com o sufixo.|
|startswith| |Devolve True se a string começar com o prefixo.|
|join| |Utiliza a string como delimitadora para concatenar uma sequência de outras strings.|
|index| |Devolve a posição do primeiro caractere de uma substring, se ela for encontrada em uma string: gera ValueError se não encontrar.|
|find| |Devolve a posição do primeiro caractere da primeira ocorrência da substring na string; é como index, porém devolve –1 se não encontrar.|
|rfind| |Devolve a posição do primeiro caractere da última ocorrência da substring na string; devolve –1 se não encontrar.|
|replace| |Substitui ocorrências de uma string por outra string.|
|strip, rstrip, lstrip| |Remove espaços em branco, incluindo quebras de linha; é equivalente a x.strip() (e a rstrip e lstrip, respectivamente) para cada elemento.|
|split| |Separa a string em uma lista de substrings usando o delimitador especificado.|
|lower| |Converte os caracteres alfabéticos para letras minúsculas.|
|upper| |Converte os caracteres alfabéticos para letras maiúsculas.|
|casefold| |Converte os caracteres para letras minúsculas e converte quaisquer combinações variáveis de caracteres específicos de região para um formato comum comparável.| 
|ljust, rjust| | Justifica à esquerda ou à direita, respectivamente; preenche o lado oposto da string com espaços (ou com outro caractere de preenchimento) para devolver uma string com um tamanho mínimo.|


### Regular Expressions

In [99]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [100]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [101]:
regex.findall(text)

['    ', '\t ', '  \t']

In [102]:
#match e search estão intimamente relacionados a findall. 
#Enquanto findall devolve todas as correspondências em uma 
#string, search devolve apenas a primeira. De modo mais 
#rigoroso, match faz a correspondência somente no início da 
#string. Como um exemplo menos trivial, vamos considerar um 
#bloco de texto e uma expressão regular capaz de 
#identificar a maioria dos endereços de email:

text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'


# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [103]:
#Usar findall no texto gera uma lista de endereços de email
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [104]:
#search devolve um objeto especial de correspondência para o 
#primeiro endereço de email no texto. Para a regex anterior, 
#o objeto de correspondência pode nos dizer apenas a posição 
#de início e de fim do padrão na string:

m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [105]:
text[m.start():m.end()]

'dave@google.com'

In [106]:
#regex.match devolve None, pois fará a correspondência 
#somente se o padrão ocorrer no início da string
print(regex.match(text))

None


In [107]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [108]:
#Suponha que quiséssemos encontrar os endereços de email e, 
#simultaneamente, segmentar cada endereço em seus 
#três componentes: nome do usuário, nome do domínio e sufixo do domínio.


pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [109]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [110]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [111]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### Vectorized String Functions in pandas

In [112]:
#Limpar um conjunto de dados desorganizado para análise 
#em geral exige muita manipulação e regularização de strings. 
#Para complicar mais ainda a situação, uma coluna contendo 
#strings ocasionalmente terá dados ausentes:

data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [113]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [114]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [115]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [116]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

## Conclusion